Step 1: Install tensorflow ranking

In [1]:
!pip install -q tensorflow-ranking
!pip install -q --upgrade tfds-nightly

     |████████████████████████████████| 141 kB 8.5 MB/s 
     |████████████████████████████████| 511.7 MB 4.1 kB/s 
     |████████████████████████████████| 1.6 MB 47.6 MB/s 
     |████████████████████████████████| 5.8 MB 28.5 MB/s 
     |████████████████████████████████| 438 kB 62.9 MB/s 
     |████████████████████████████████| 4.5 MB 9.3 MB/s 


Step 2: Import tensorflow ranking and tensorflow datasets.

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_ranking as tfr

Step 3: Importing Dataset

In [3]:
# Load MSLR-WEB10K dataset from TFDS.
ds = tfds.load("mslr_web/30k_fold1", split="train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/18919 [00:00<?, ? examples/s]

...

Generating vali examples...:   0%|          | 0/6306 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/mslr_web/30k_fold1/1.0.0.incompleteQJA47G/mslr_web-vali.tfrecord*...:   0%…

Generating test examples...:   0%|          | 0/6306 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/mslr_web/30k_fold1/1.0.0.incompleteQJA47G/mslr_web-test.tfrecord*...:   0%…

Dataset mslr_web downloaded and prepared to /root/tensorflow_datasets/mslr_web/30k_fold1/1.0.0. Subsequent calls will reuse this data.


Step 4: Adding mask tensor

In [4]:
# Add a mask tensor.
ds = ds.map(lambda feature_map:{"_mask": tf.ones_like(feature_map["label"], dtype=tf.bool),    **feature_map})

Step 5: Shuffle data & create padded batches

In [5]:
# Shuffle data and create padded batches for queries with different lengths.
# Padded items will have False on `_mask`.
ds = ds.shuffle(buffer_size=1000).padded_batch(batch_size=32)

Step 6: Create Tuples

In [6]:
# Create (features, labels) tuples from data and set -1 label for masked items.
ds = ds.map (lambda 
feature_map:(feature_map, tf.where(feature_map["_mask"], feature_map.pop("label"),-1.)))

Step 7: Creating a model with Keras Functional API

In [7]:
# Create a model with Keras Functional API.
inputs = {    
    name: tf.keras.Input(shape=(None,1), dtype=tf.float32, name=name)
for name in ds.element_spec[0]
if name != "_mask"}
norm_inputs = [tf.keras.layers.BatchNormalization()(x)for x in inputs.values()]
x = tf.concat(norm_inputs, axis=-1)
for layer_width in[128,64,32]:  
  x = tf.keras.layers.Dense(units=layer_width)(x)  
  x = tf.keras.layers.Activation(activation=tf.nn.relu)(x)
  scores = tf.squeeze(tf.keras.layers.Dense(units=1)(x), axis=-1)

Step 8: Creating & Compiling & Training the models

In [8]:
# Create, compile and train the model.
model = tf.keras.Model(inputs=inputs, outputs=scores)
model.compile(    
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),    
    loss=tfr.keras.losses.SoftmaxLoss(),    
    metrics=tfr.keras.metrics.get("ndcg", topn=5, name="NDCG@5"))
model.fit(ds, epochs=3)

Epoch 1/3


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['_mask'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


592/592 [==============================] - 128s 169ms/step - loss: 402.6566 - NDCG@5: 0.3800
Epoch 2/3
592/592 [==============================] - 101s 168ms/step - loss: 401.7350 - NDCG@5: 0.4109
Epoch 3/3
592/592 [==============================] - 108s 179ms/step - loss: 401.4805 - NDCG@5: 0.4176
